# Building Applications with LLMs

- [Skool Link](https://www.skool.com/data-alchemy/classroom/a455582e?md=cffccd9c21af41e0bc3669610ce3bc39)
- [YouTube](https://www.youtube.com/watch?time_continue=1783&v=NYSWn1ipbgg&embeds_referring_euri=https%3A%2F%2Fwww.skool.com%2F&embeds_referring_origin=https%3A%2F%2Fwww.skool.com&source_ve_path=Mjg2NjY&feature=emb_logo)
- [LangChain Experiments](https://github.com/daveebbelaar/langchain-experiments)

In [1]:
import os
import dotenv
import shelve
import time

dotenv_path = dotenv.find_dotenv()
dotenv.load_dotenv(dotenv_path)

True

In [2]:
from openai import OpenAI
client = OpenAI()

In [3]:
# --------------------------------------------------------------
# Thread management
# --------------------------------------------------------------
def check_if_thread_exists(wa_id):
    with shelve.open("threads_db") as threads_shelf:
        return threads_shelf.get(wa_id, None)


def store_thread(wa_id, thread_id):
    with shelve.open("threads_db", writeback=True) as threads_shelf:
        threads_shelf[wa_id] = thread_id


In [4]:
# --------------------------------------------------------------
# Run assistant
# --------------------------------------------------------------
def run_assistant(thread):
    # Retrieve the Assistant
    assistant = client.beta.assistants.retrieve("asst_Y9BVXJgGAd15G9wclbapzIhT")

    # Run the assistant
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )

    # Wait for completion
    while run.status != "completed":
        # Be nice to the API
        time.sleep(0.5)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    # Retrieve the Messages
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    new_message = messages.data[0].content[0].text.value
    print(f"Generated message: {new_message}")
    return new_message

In [5]:
# --------------------------------------------------------------
# Generate response
# --------------------------------------------------------------
def generate_response(message_body, wa_id, name):
    # Check if there is already a thread_id for the wa_id
    thread_id = check_if_thread_exists(wa_id)

    # If a thread doesn't exist, create one and store it
    if thread_id is None:
        print(f"Creating new thread for {name} with wa_id {wa_id}")
        thread = client.beta.threads.create()
        store_thread(wa_id, thread.id)
        thread_id = thread.id

    # Otherwise, retrieve the existing thread
    else:
        print(f"Retrieving existing thread for {name} with wa_id {wa_id}")
        thread = client.beta.threads.retrieve(thread_id)

    # Add message to thread
    message = client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=message_body,
    )

    # Run the assistant and get the new message
    new_message = run_assistant(thread)
    print(f"To {name}:", new_message)
    return new_message

In [6]:

# --------------------------------------------------------------
# Test assistant
# --------------------------------------------------------------

new_message = generate_response("When will Airbnb collect OccupancyTax?", "123", "John")
print(new_message)

Retrieving existing thread for John with wa_id 123
Generated message: Airbnb will begin collecting and remitting Occupancy Tax on March 1, 2017. From this date forward, Airbnb will handle the Occupancy taxes for all bookings on their platform, and as a host or guest, you will not be responsible for collecting such taxes . Therefore, if your stay is after March 1, 2017, the tax should have been collected by Airbnb at the time of booking.
To John: Airbnb will begin collecting and remitting Occupancy Tax on March 1, 2017. From this date forward, Airbnb will handle the Occupancy taxes for all bookings on their platform, and as a host or guest, you will not be responsible for collecting such taxes . Therefore, if your stay is after March 1, 2017, the tax should have been collected by Airbnb at the time of booking.
Airbnb will begin collecting and remitting Occupancy Tax on March 1, 2017. From this date forward, Airbnb will handle the Occupancy taxes for all bookings on their platform, and a